# 附录 10.1：链式提示

- [课程](#lesson)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格，以加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# 已重写为接受任意长度的消息列表
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## 课程

俗话说，“写作就是重写。” 事实证明，**当被要求时，Claude 通常可以提高其响应的准确性**！

有许多方法可以提示 Claude “再思考一次”。那些向人类请求再次核对工作时感觉自然的方式，通常也适用于 Claude。（请查看我们的[提示链文档](https://docs.anthropic.com/claude/docs/chain-prompts)，了解何时以及如何使用提示链的更多示例。）

### 示例

在这个例子中，我们要求 Claude 想出十个单词……但其中一个或多个不是真正的单词。

In [ ]:
# 初始提示
first_user = "说出十个以字母 'ab' 结尾的单词。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印 Claude 的响应
first_response = get_completion(messages)
print(first_response)

**要求 Claude 提高其回答的准确性**可以修复错误！

下面，我们已经将 Claude 之前不正确的响应提取出来，并在对话中增加了另一个回合，要求 Claude 修正其先前的答案。

In [ ]:
second_user = "请找出所有非真实单词的替代词。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印 Claude 的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(messages))

但是，Claude 仅仅因为我们告诉它这样做就修改答案了吗？如果我们一开始就给出一个正确答案呢？Claude 会失去信心吗？在这里，我们用一个正确响应替换了 `first_response`，并要求它再次核对。

In [ ]:
first_user = "说出十个以字母 'ab' 结尾的单词。"

first_response = """以下是 10 个以字母 'ab' 结尾的单词：

1. Cab (出租车)
2. Dab (轻拍)
3. Grab (抓取)
4. Gab (闲聊)
5. Jab (猛刺)
6. Lab (实验室)
7. Nab (逮捕)
8. Slab (厚板)
9. Tab (标签)
10. Blab (泄密)"""

second_user = "请找出所有非真实单词的替代词。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印 Claude 的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(messages))

你可能会注意到，如果你多次从上面的代码块生成响应，Claude 大部分时间会保持单词不变，但偶尔仍然会更改这些单词，即使它们都已经正确。我们能做些什么来缓解这种情况呢？根据第八章，我们可以给 Claude 一个“退路”！让我们再试一次。

In [ ]:
first_user = "说出十个以字母 'ab' 结尾的单词。"

first_response = """以下是 10 个以字母 'ab' 结尾的单词：

1. Cab (出租车)
2. Dab (轻拍)
3. Grab (抓取)
4. Gab (闲聊)
5. Jab (猛刺)
6. Lab (实验室)
7. Nab (逮捕)
8. Slab (厚板)
9. Tab (标签)
10. Blab (泄密)"""

second_user = "请找出所有非真实单词的替代词。如果所有单词都是真实单词，则返回原始列表。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印 Claude 的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(messages))

尝试从上面的代码多次生成响应，你会发现 Claude 现在更善于坚持自己的立场了。

你也可以使用提示链来**要求 Claude 改进其响应**。下面，我们要求 Claude 先写一个故事，然后改进它自己写的故事。你的个人品味可能有所不同，但许多人可能会同意 Claude 的第二个版本更好。

首先，让我们生成 Claude 故事的第一个版本。

In [ ]:
# 初始提示
first_user = "写一个关于一个喜欢跑步的女孩的三句话短篇故事。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# 存储并打印 Claude 的响应
first_response = get_completion(messages)
print(first_response)

现在，让我们让 Claude 改进它的初稿。

In [ ]:
second_user = "把故事写得更好。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印 Claude 的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(messages))

这种形式的替换非常强大。我们一直在使用替换占位符来传入列表、单词、Claude 之前的响应等等。你还可以**使用替换来做我们称之为“函数调用”的事情，即要求 Claude 执行某个函数，然后将该函数的结果作为输入，要求 Claude 进一步处理这些结果**。它的工作方式与任何其他替换一样。更多内容将在下一个附录中介绍。

下面是另一个将对 Claude 的一次调用结果插入到另一次更长的调用中的示例。让我们从第一个提示开始（这次包括预填充 Claude 的响应）。

In [ ]:
first_user = """从以下文本中找出所有名字：

"嘿，杰西。是我，艾琳。我打电话是关于乔伊明天要办的派对。凯莎说她会来，我想梅尔也会在那里。"""

prefill = "<names>"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# 存储并打印 Claude 的响应
first_response = get_completion(messages)
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(first_response)

让我们把这个名字列表传入另一个提示。

In [ ]:
second_user = "将列表按字母顺序排列。"

# API 消息数组
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# 打印 Claude 的响应
print("------------------------ 包含变量替换的完整消息数组 ------------------------")
print(messages)
print("\n------------------------------------- Claude 的响应 -------------------------------------")
print(get_completion(messages))

既然你已经了解了提示链，请前往附录 10.2 学习如何使用提示链实现函数调用。

---

# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Claude to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

**Asking Claude to make its answer more accurate** fixes the error! 

Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

But is Claude revising its answer just because we told it to? What if we start off with a correct answer already? Will Claude lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

You may notice that if you generate a respnse from the above block a few times, Claude leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Claude an out! Let's try this one more time.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Try generating responses from the above code a few times to see that Claude is much better at sticking to its guns now.

You can also use prompt chaining to **ask Claude to make its responses better**. Below, we asked Claude to first write a story, and then improve the story it wrote. Your personal tastes may vary, but many might agree that Claude's second version is better.

First, let's generate Claude's first version of the story.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Now let's have Claude improve on its first draft.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, Claude's former responses, and so on. You can also **use substitution to do what we call "function calling," which is asking Claude to perform some function, and then taking the results of that function and asking Claude to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is one more example of taking the results of one call to Claude and plugging it into another, longer call. Let's start with the first prompt (which includes prefilling Claude's response this time).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

Let's pass this list of names into another prompt.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))